In [14]:
import cv2
import numpy as np
import os

# Open all the video folders to find the paths
video_folder = "/home/persie/Videos/mokap/"
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))

def get_frame(video_path: int, frame_index: int) -> None:
    vid = cv2.VideoCapture(video_path)
    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = vid.read()
    vid.release()
    if not ret:
        return None
    else:
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
def draw_circle(event,x,y,flags,param):
    #Because of the use of global variables, they must be defined before using them, outside of any functions
    global radius, text_image, mask  
    if flags == cv2.EVENT_FLAG_SHIFTKEY:
            cv2.circle(mask,(x,y),radius,(0,0,0),-1)    
    if flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(mask,(x,y),radius,(255,255,255),-1)      
        
    text_image = cv2.rectangle(text_image, (0, 70), (35, 40), (255, 255, 255), -1)    
    text_image = cv2.putText(text_image, str(radius), (0, 65), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(0, 0, 255), thickness=2)  

#Find the moment of the reduced mask
def find_centre(contours):
    
    # Calculate image moments of the detected contour
    M = cv2.moments(contours[0])
    
    # Print center (debugging):
    x = round(M['m10'] / M['m00'])
    y = round(M['m01'] / M['m00'])
    
    return x,y
        
def create_bgs_short(clearest_frame):
    back_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=9, detectShadows=True)
    
    back_sub.setShadowThreshold(0.49)
    
    back_sub.apply(clearest_frame, None, 1)
  
    return back_sub

def create_bgs_long(video_path, sample_rate=50):

    cap = cv2.VideoCapture(video_path)    
    # Read the first frame, but discard as first frame is blank
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    back_sub = cv2.createBackgroundSubtractorMOG2(history=1000, varThreshold=11, detectShadows=True)
    back_sub.setShadowThreshold(0.049)
    
    fno = 1
    while ret:
        if fno % sample_rate == 0:
            ret, frame = cap.retrieve()
            if not ret:
                break
            # Convert current frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            back_sub.apply(gray, None, 0.1)
            fno += 1
        else:
            # read next frame
            ret = cap.grab()
            fno += 1
    return back_sub
  
    




In [26]:
def find_convex_hull(image, mask):
    #Attempt to refine the mask
    #Create an image where every pixel outside of the mask is set to the mean value of the image outside the mask
    thresh = cv2.mean(image, mask=cv2.bitwise_not(mask))[0]
    average_bg = cv2.bitwise_and(cv2.bitwise_not(mask), thresh)
    masked_seed = cv2.bitwise_and(image, image, mask=mask)
    average_image = cv2.bitwise_xor(masked_seed, average_bg)
    
    #Blur image and apply adaptive thresholding
    blur = cv2.GaussianBlur(average_image,(7,7),0)
    _, thresh_image = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    blur_thresh = cv2.blur(thresh_image, (3,3))
    contours,_ = cv2.findContours(blur_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #Create an empty array of edges to be filled
    edges = np.empty((0,2), dtype=np.uint8)
    
    #Compile all contours in image but ignore lines that are shorter than the threshold
    for i in range(len(contours)):
        if len(contours[i]) > 15:
            edges = np.concatenate([edges, *contours[i]])
    
    new_mask  = np.zeros_like(image, dtype=np.uint8)
    
    try:
        convex_hull = cv2.convexHull(edges)
    except Exception as e: 
        print(e)
        return None 
    
    cv2.drawContours(new_mask, [convex_hull], -1, (255,255,255), -1)
    x,y = find_centre([convex_hull]) #This is called 39 times and I don't know how to stop it
    return convex_hull[0], new_mask, x, y


def find_bounding_box(contours, image, mask):    
    x,y,w,h = cv2.boundingRect(contours)
    template = image[y:y+h,x:x+w]
    cropped_mask = mask[y:y+h,x:x+w]
    return template, cropped_mask

def match_template(image, template, mask):
    # Apply template Matching
    res = cv2.matchTemplate(image,template, cv2.TM_CCORR, mask=mask)
    _, max_val, _, max_loc = cv2.minMaxLoc(res)
    #top_left = max_loc
    x,y = max_loc
    return x,y, max_val

def transform_mask(mask, old_x, old_y, new_x, new_y):
    dx = new_x - old_x
    dy = new_y - old_y
    T = np.float32([[1,0,dx],[0,1,dy]])
    new_mask = cv2.warpAffine(mask,T, (mask.shape[1], mask.shape[0]))
    
    return new_mask
    

In [27]:
help(cv2.minMaxLoc)

Help on built-in function minMaxLoc:

minMaxLoc(...)
    minMaxLoc(src[, mask]) -> minVal, maxVal, minLoc, maxLoc
    .   @brief Finds the global minimum and maximum in an array.
    .   
    .   The function cv::minMaxLoc finds the minimum and maximum element values and their positions. The
    .   extremums are searched across the whole array or, if mask is not an empty array, in the specified
    .   array region.
    .   
    .   The function do not work with multi-channel arrays. If you need to find minimum or maximum
    .   elements across all the channels, use Mat::reshape first to reinterpret the array as
    .   single-channel. Or you may extract the particular channel using either extractImageCOI, or
    .   mixChannels, or split.
    .   @param src input single-channel array.
    .   @param minVal pointer to the returned minimum value; NULL is used if not required.
    .   @param maxVal pointer to the returned maximum value; NULL is used if not required.
    .   @param minL

In [23]:
# For video 33 (for example) select a clear frame to use as the background
video_id = 68
print(video_file[video_id])
clear_frame_id = 10
object_frame_id = 1507

/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4


In [16]:
clear_frame = get_frame(video_file[video_id], clear_frame_id)
object_frame = get_frame(video_file[video_id], object_frame_id)

In [17]:
# Create a mask of the object being tracked, and scan it for good features to track
# Run the mask creation code
text_image = object_frame.copy()
mask = np.zeros_like(object_frame, np.uint8)
# Set the initial circle radius
radius = 50
line_1 = "Click with the left mouse button to add, double click to remove"
line_2 = "Up = radius + 5,"
line_3 = "Down = radius - 5"
text_image = cv2.putText(text_image, line_1, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
text_image = cv2.putText(text_image, line_2, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
text_image = cv2.putText(text_image, line_3, (10, 34), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
combined_image = cv2.bitwise_or(text_image, mask)
cv2.namedWindow('Mask On Frame')
cv2.imshow('Mask On Frame', combined_image)
cv2.setMouseCallback('Mask On Frame',draw_circle)


while(1):
    combined_image = cv2.bitwise_or(text_image, mask)
    cv2.imshow('Mask On Frame',combined_image)
    cv2.imshow('Mask',mask)
    k = cv2.waitKey(20) & 0xFF
    if k == ord("q"):
        break
    if k == 82:
        #upkey
        radius = radius + 5
    if k == 84:
        #downkey
        radius = radius - 5

cv2.destroyAllWindows()

In [18]:
#Save the mask if you like it
object_mask = mask

In [19]:
# Create a background subtractor with the clear frame
bgs = create_bgs_short(clear_frame)

In [24]:
# Set the initial frame and mask
initial_frame = 1

convex_hull, convex_mask, x, y = find_convex_hull(object_frame, object_mask)
#Crop the mask for the first time
template, cropped_mask = find_bounding_box(convex_hull, object_frame, convex_mask)

cap = cv2.VideoCapture(video_file[video_id])
cap.set(cv2.CAP_PROP_POS_FRAMES, initial_frame)

template_match_threshold = 0.8

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Find the pixel co-ordinates of the new match
    new_x, new_y, max_template_match = match_template(gray, template, cropped_mask)
    
    if max_template_match > template_match_threshold:    
        #Move the mask to the new match
        new_mask = transform_mask(mask, x, y, new_x, new_y)
    else:
        #Leave the template where it is
        new_mask = mask
    
    cv2.imshow('New Mask', new_mask)
    #Inflate the mask to allow for movement
    new_mask = cv2.dilate(new_mask, None, iterations=2)
    cv2.imshow('New Mask (D)', new_mask)
    #Find the convex hull
    convex_hull, convex_mask, _, _ = find_convex_hull(gray, new_mask)
    cv2.imshow('Convex Mask', convex_mask)
    #Crop the mask
    new_template, cropped_mask = find_bounding_box(convex_hull, gray, convex_mask)

    #Plot the convex hull on a temporary image frame
    new_f = np.zeros_like(gray, np.uint8)
    cv2.drawContours(new_f, [convex_hull], 0, (255, 255, 255), -1)

    cv2.imshow('Output', gray)
    cv2.imshow('Contour', new_f)

    #Reset held values
    x = new_x
    y = new_y
    template = new_template
    mask = convex_mask

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



728
588
426
756
1064
783
1064
783
1064
783
1064
784


error: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1729143277002/work/modules/imgproc/src/convhull.cpp:143: error: (-215:Assertion failed) total >= 0 && (depth == CV_32F || depth == CV_32S) in function 'convexHull'


In [ ]:
#Find initial moment
contours, hierarchy = cv2.findContours(object_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnt = contours[0]
x, y, w, h = cv2.boundingRect(cnt)

#Attempt to refine the mask
thresh = cv2.mean(object_frame, mask=cv2.bitwise_not(object_mask))[0]
average_bg = cv2.bitwise_and(cv2.bitwise_not(object_mask), thresh)
masked_seed = cv2.bitwise_and(object_frame, object_frame, mask=object_mask)
thresh_bg = np.ones_like(object_frame) * np.ceil(thresh).astype(np.uint8)
average_image = cv2.bitwise_xor(masked_seed, average_bg)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(average_image, (7, 7), 0)
ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow("blended", average_image)
cv2.imshow("th3", th3)

src = th3.copy()

# Create Window
source_window = 'Source'
cv2.namedWindow(source_window, cv2.WINDOW_NORMAL)

cv2.imshow('Source', src)
color = []
for n in range(0, 20):
    color.append((rng.randint(0, 256), rng.randint(0, 256), rng.randint(0, 256)))

    blur_fg = cv2.blur(src, (3, 3))
    # Find contours
    contours, _ = cv2.findContours(blur_fg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    edges = np.empty((0, 2), dtype=np.uint8)

    #Find the overall contour
    for i in range(len(contours)):
        if len(contours[i]) > 15:
            edges = np.concatenate([edges, *contours[i]])

    convex_hull = cv2.convexHull(edges)
    drawing = np.zeros_like(src, dtype=np.uint8)
    new_mask = np.zeros_like(src, dtype=np.uint8)
    new_edge = object_frame.copy()
    cv2.drawContours(drawing, [convex_hull], -1, color[0], 3)
    cv2.drawContours(new_edge, [convex_hull], -1, color[0], 3)
    cv2.drawContours(new_mask, [convex_hull], -1, (255, 255, 255), -1)

    x, y = find_centre([convex_hull])  #This is called 39 times and I don't know how to stop it
    cv2.circle(new_mask, (x, y), 5, (0, 255, 0), -1)

while True:
    # Show in a window
    cv2.imshow('Contours', drawing)
    cv2.imshow('New', new_edge)
    cv2.imshow('New Mask', new_mask)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#Apply an affine warp to the mask
affine = cv2.getAffineTransform([cv2.KeyPoint[0, 0], [0, 5], [5, 0]], [[45, 45], [45, 50], [50, 45]])

out = cv2.warpAffine(new_edge, affine, (new_mask.shape[1], new_mask.shape[0]))

while True:
    cv2.imshow('New Mask', new_mask)
    cv2.imshow('Transformed Mask', out)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [25]:
cap.release()
cv2.destroyAllWindows()